In [15]:
import pandas as pd
import json
import os
from tqdm.auto import tqdm

from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
VSE_GPT_API_KEY = ""
os.environ["OPENAI_API_KEY"] = VSE_GPT_API_KEY

In [3]:
with open('/Users/alfa/Code/financial_assistant/data/interim/test_qual_investor/questions.json', 'r') as f:
    questions = json.load(f)

In [4]:
with open('/Users/alfa/Code/financial_assistant/data/interim/test_qual_investor/chapters.json', 'r') as f:
    chapters = json.load(f)

In [21]:
norm_zs = pd.read_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o_zeroshot.csv', index_col=0)
mini_zs = pd.read_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o-mini_zeroshot.csv', index_col=0)

norm_rag = pd.read_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o_rag.csv', index_col=0)
mini_rag = pd.read_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o-mini_rag.csv', index_col=0)

In [37]:
norm_zs.options = norm_zs.options.apply(eval)
mini_zs.options = mini_zs.options.apply(eval)
norm_rag.options = norm_rag.options.apply(eval)
mini_rag.options = mini_rag.options.apply(eval)

In [38]:
norm_zs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1.4 to 11.7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question       44 non-null     object
 1   options        44 non-null     object
 2   answer         44 non-null     object
 3   chapter        44 non-null     int64 
 4   llm_answer     44 non-null     object
 5   llm_reasoning  44 non-null     object
 6   correct        44 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 3.8+ KB


# Metrics

In [72]:
norm_zs.correct.mean()

0.9318181818181818

In [73]:
norm_rag.correct.mean()

0.9545454545454546

In [74]:
mini_zs.correct.mean()

0.7727272727272727

In [76]:
mini_rag.correct.mean()

0.9545454545454546

# Check different

In [23]:
norm_rag[(norm_zs.correct==0) & (norm_rag.correct==1)]

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
2.5,Если инвестор принимает решение продать принад...,"[{'letter': 'А', 'option_text': 'В любой рабоч...",Б,2,Б,"Согласно контексту, сделка с акциями может быт...",1
4.6,Вы являетесь покупателем по первой части догов...,"[{'letter': 'А', 'option_text': 'В любом случа...",А,4,А,"Согласно контексту, дивиденды, купоны и другие...",1


In [24]:
norm_zs[(norm_zs.correct==0) & (norm_rag.correct==1)]

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
2.5,Если инвестор принимает решение продать принад...,"[{'letter': 'А', 'option_text': 'В любой рабоч...",Б,2,А,На российском фондовом рынке акции можно прода...,0
4.6,Вы являетесь покупателем по первой части догов...,"[{'letter': 'А', 'option_text': 'В любом случа...",А,4,Б,Согласно российскому законодательству и практи...,0


In [25]:
mini_rag[(mini_zs.correct==0) & (mini_rag.correct==1)]

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
1.5,Что из перечисленного не является риском по пр...,"[{'letter': 'А', 'option_text': 'Риск изменени...",А,1,А,Изменение суверенного рейтинга России не влияе...,1
2.5,Если инвестор принимает решение продать принад...,"[{'letter': 'А', 'option_text': 'В любой рабоч...",Б,2,Б,"Инвестор может продать акцию, но точная дата с...",1
2.7,Выберите верное утверждение,"[{'letter': 'А', 'option_text': 'Акция, не вкл...",А,2,А,"Согласно контексту, акция, не включённая в кот...",1
4.6,Вы являетесь покупателем по первой части догов...,"[{'letter': 'А', 'option_text': 'В любом случа...",А,4,А,"Согласно контексту, дивиденды и другие доходы ...",1
6.5,Может ли инвестор по структурной облигации при...,"[{'letter': 'А', 'option_text': 'Да, может.'},...",А,6,А,Инвестор по структурной облигации может получи...,1
7.7,Допускается ли вторичное обращение паев закрыт...,"[{'letter': 'А', 'option_text': 'Да, если это ...",А,7,А,Вторичное обращение паев закрытого паевого инв...,1
9.7,"В случае, если Вы купили иностранную облигацию...","[{'letter': 'А', 'option_text': '10 долларов С...",В,9,В,Ваш налогооблагаемый доход в России будет расс...,1
11.4,Выберите правильное утверждение в отношении па...,"[{'letter': 'А', 'option_text': 'Инвестиции в ...",В,11,В,Инвестиции в ETF на индекс акций обеспечивают ...,1


In [18]:
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    openai_api_base = "https://api.vsegpt.ru/v1/"
)

In [19]:
db = FAISS.load_local("/Users/alfa/Code/financial_assistant/data/processed/docs_db_index", embedding_model)

In [56]:
def get_context(
    question_dict
):
    
    options = [
        opt["letter"] + '. ' + opt["option_text"]
        for opt in question_dict["options"]
    ]

    question_with_options_text = '\n'.join([question_dict['question']] + options)

    top_docs = db.similarity_search(question_with_options_text, 3)
    top_docs_texts = [f"{i}. {doc.page_content}" for i, doc in enumerate(top_docs)]
    context = '\n'.join(top_docs_texts)
    return question_with_options_text, context

In [40]:
norm_zs.loc[2.5].to_dict()

{'question': 'Если инвестор принимает решение продать принадлежащую ему акцию, как быстро он может это сделать?',
 'options': [{'letter': 'А',
   'option_text': 'В любой рабочий день, определенный самим инвестором.'},
  {'letter': 'Б', 'option_text': 'Точная дата не может быть определена.'},
  {'letter': 'В', 'option_text': 'В течение недели.'},
  {'letter': 'Г', 'option_text': 'В течение 7 дней.'}],
 'answer': 'Б',
 'chapter': 2,
 'llm_answer': 'А',
 'llm_reasoning': 'На российском фондовом рынке акции можно продать в любой рабочий день, когда биржа открыта, что определяется графиком работы биржи, а не самим инвестором.',
 'correct': 0}

In [57]:
options, context = get_context(norm_zs.loc[2.5].to_dict())

In [63]:
print(f'Вопрос:\n{options}')
print(f'\nКонтекст:\n{context}\n')
print('\nВерный ответ:', norm_zs.loc[2.5].to_dict()['answer'])
print('Ответ без контекста:', norm_zs.loc[2.5].to_dict()['llm_answer'])
print('Ответ c контекстом:', norm_rag.loc[2.5].to_dict()['llm_answer'])
print('\nОбоснование без контекста:', norm_zs.loc[2.5].to_dict()['llm_reasoning'])
print('Обоснование c контекстом:', norm_rag.loc[2.5].to_dict()['llm_reasoning'])

Вопрос:
Если инвестор принимает решение продать принадлежащую ему акцию, как быстро он может это сделать?
А. В любой рабочий день, определенный самим инвестором.
Б. Точная дата не может быть определена.
В. В течение недели.
Г. В течение 7 дней.

Контекст:
0. Инвестор может совершить сделку с акциями, только когда на бирже найдётся подходящий контрагент. Поэтому точную дату и время совершения сделки предсказать невозможно. При этом если инвестор захочет продать иностранные акции, он может столкнуться с техническими проблемами из-за риска введения иностранным регулятором ограничений на совершение сделок. Многие инвесторы предпочитают делать вложения в акции различных компаний, так как они в перспективе могут принести неограниченный доход. Но акции также могут падать в цене и приводить к финансовым потерям.
1. В дату оферты инвестор может предъявить свои облигации к погашению до даты погашения всего выпуска, но это его право, а не обязанность.
2. ☝️ Чтобы попасть в реестр и получить дивид

In [64]:
options, context = get_context(norm_zs.loc[4.6].to_dict())

In [65]:
print(f'Вопрос:\n{options}')
print(f'\nКонтекст:\n{context}\n')
print('\nВерный ответ:', norm_zs.loc[4.6].to_dict()['answer'])
print('Ответ без контекста:', norm_zs.loc[4.6].to_dict()['llm_answer'])
print('Ответ c контекстом:', norm_rag.loc[4.6].to_dict()['llm_answer'])
print('\nОбоснование без контекста:', norm_zs.loc[4.6].to_dict()['llm_reasoning'])
print('Обоснование c контекстом:', norm_rag.loc[4.6].to_dict()['llm_reasoning'])

Вопрос:
Вы являетесь покупателем по первой части договора репо. По ценным бумагам, которые Вы получили по договору репо, осуществлена выплата денежных средств или передано иное имущество, в том числе в виде дивидендов или процентов (доход). В каком случае Вы обязаны передать сумму такого дохода продавцу по договору репо?
А. В любом случае, даже если Вы этот доход фактически не получили.
Б. Вы не обязаны передавать продавцу по договору репо такой доход, если это предусмотрено условиями договора репо.
В. Только если Вы являетесь эмитентом ценных бумаг, приобретенных по договору репо.
Г. Ни один из ответов не является правильным.

Контекст:
0. Если за время между первой и второй частями сделки на ценные бумаги будут начислены дивиденды, купонные выплаты или процентный доход — они по закону попадут к текущему владельцу бумаг. При этом он обязан передать все полученные доходы продавцу по первой части сделки, даже если этот доход ещё фактически не получен. Как и когда — определяется в их дог